In [ ]:
import torch
import os
import yaml
from yacs.config import CfgNode as CN
from fvcore.nn import FlopCountAnalysis
from flops import calculate_flops_and_time
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [32]:
from dataloader import build_dataloader
from networks import build_model
from trainers import AverageMeter

In [ ]:
# TODO: change these
root = '/home/ray/proj/IDL_11785_Project/configs/'
model_name = 'cvt'
model_architecture = 'cvt_13_224.yaml'
model_checkpoint = '/home/ray/proj/IDL_11785_Project/checkpoints/cvt/test1/model_best.pth'

In [40]:
config_path = os.path.join(root, model_name, model_architecture)
with open(config_path, 'r') as file:
    cfg = CN(yaml.safe_load(file))

### Dataset

In [17]:
test_loader = build_dataloader(cfg.model.name, cfg, is_train=False, is_test=False)

Transform = 
Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
CenterCrop(size=(224, 224))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet100
building dataset data/imagenet100/val_data


In [ ]:
print('number of batches:', len(test_loader))
for image, target in test_loader:
    print('image shape:', image.shape)
    print('target shape:', target.shape)
    break

number of batches: 157
image shape: torch.Size([32, 3, 224, 224])
target shape: torch.Size([32])
output shape: torch.Size([32, 100])


### Model

In [ ]:
from torchinfo import summary
model = build_model(cfg.model.architecture).to(device)
summary(model, input_size=(1, 3, cfg.image_size[0], cfg.image_size[0]))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [ ]:
checkpoint = torch.load(model_checkpoint, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

### Accuracy

In [34]:
@torch.no_grad()
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    if isinstance(output, list):
        output = output[-1]

    maxk = max(topk)
    batch_size = target.size(0)

    if target.ndim > 1:  # Check if target is one-hot encoded
        target = target.argmax(dim=1)
    
    #print(target)
    #print(output)
    #predictions = output.argmax(dim=1)
    #print(predictions)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size).item())
    return res


In [37]:
@torch.no_grad
def evaluate(model, data_loader):
    top1 = AverageMeter()
    top5 = AverageMeter()

    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, position=0, leave=False, desc='Validating')
    for i, (x, y) in enumerate(data_loader):
        # compute output
        #x = x.cuda(non_blocking=True)
        #y = y.cuda(non_blocking=True)
        outputs = model(x)

        prec1, prec5 = accuracy(outputs, y, (1, 5))

        top1.update(prec1, x.size(0))
        top5.update(prec5, x.size(0))
        
        batch_bar.set_postfix(
            top1="{:.04f}".format(top1.avg), 
            top5="{:.04f}".format(top5.avg)
        )
        batch_bar.update()

        torch.cuda.synchronize()
    return top1.avg, top5.avg

In [ ]:
top1_acc, top5_acc = evaluate(model, test_loader)
print('Top-1 accuracy:', top1_acc)
print('Top-5 accuracy:', top5_acc)

Validating:   0%|          | 0/157 [00:00<?, ?it/s]

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

### FLOPs

In [ ]:
dummy_input = torch.rand((1, 3, cfg.image_size[0], cfg.image_size[0]))
calculate_flops_and_time(model, dummy_input)

In [36]:
flops = FlopCountAnalysis(model, image)
flops_total = flops.total()
flops_module_operator = flops.by_module_and_operator() 
print(f"=>flops total '{flops_total}', flops_module_operator {flops_module_operator}")

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor